In [3]:
import os

import ppxf

### Verification of models in pPXF

In [4]:
ppxf_dir = os.path.split(ppxf.__file__)[0]
miles_dir = os.path.join(ppxf_dir, "miles_models")

In [5]:
miles_models = [_ for _ in os.listdir(miles_dir) if _.startswith("Mun")]

In [6]:
miles_models[:10]

['Mun1.30Zp0.00T00.7943_iPp0.00_baseFe_linear_FWHM_2.51.fits',
 'Mun1.30Zm1.31T00.3981_iPp0.00_baseFe_linear_FWHM_2.51.fits',
 'Mun1.30Zm0.71T01.0000_iPp0.00_baseFe_linear_FWHM_2.51.fits',
 'Mun1.30Zp0.22T00.2512_iPp0.00_baseFe_linear_FWHM_2.51.fits',
 'Mun1.30Zp0.22T15.8489_iPp0.00_baseFe_linear_FWHM_2.51.fits',
 'Mun1.30Zm0.40T01.9953_iPp0.00_baseFe_linear_FWHM_2.51.fits',
 'Mun1.30Zm1.71T00.1585_iPp0.00_baseFe_linear_FWHM_2.51.fits',
 'Mun1.30Zm0.40T01.5849_iPp0.00_baseFe_linear_FWHM_2.51.fits',
 'Mun1.30Zm1.31T00.1000_iPp0.00_baseFe_linear_FWHM_2.51.fits',
 'Mun1.30Zp0.22T01.0000_iPp0.00_baseFe_linear_FWHM_2.51.fits']

In the Miles models, we can infer the type of models using the [SEDs name convention](http://research.iac.es/proyecto/miles/pages/ssp-models/name-convention.php). In particular, we understand that the models that come with pPXF have the Miles spectral range, unimodel IMF with log slope of -1.3 (Mun1.30...). We can also see that it has a metallicity Z=0 in the first file, so so it has Padova isochrone. Now, the particular set of ages and metallicities can be infered by inspecting the name of all the files used.

In [7]:
len(miles_models)

150

In [8]:
Zs = set([float(_.split("Z")[1].split("T")[0].replace("m", "-").replace("p", "+")) for _ in miles_models])
Zs

{-1.71, -1.31, -0.71, -0.4, 0.0, 0.22}

In [9]:
Ts = set([float(_.split("T")[1].split("_iP")[0]) for _ in miles_models])
print("Number of ages in grid: ", len(Ts))
print(min(Ts), max(Ts))

Number of ages in grid:  25
0.0631 15.8489


So, the models assume 25 different ages and 6 metallicities, resulting in 150 models. 

### Changing the stellar population models to BaSTI isochrones

If we are interested in [alpha/Fe], we should use models that are produced with BaSTI isochrones, as they provide models with [alpha/Fe]=0.0 and [alpha/Fe]=0.4 (all models from Padova have solar alpha-abundance). Miles models can be obtained directly from the [tune SSP models](http://research.iac.es/proyecto/miles/pages/webtools/tune-ssp-models.php). From this website, we see that BaSTI have a larger grid of models: 12 metallicities, 53 ages and 3 alpha-element abundances (Ep0.00 and Ep0.40, ignore base models). 

In [10]:
n_models = 12 * 53 * 2
n_models

1272

To replace the models in pPXF, you can simply download the models using the link above, unpack the spectra in any directory, and point out the Python code to that location instead of the miles_dir above. The same models can be used with paintbox.

In theory, pPXF could use all models to get the solution, but it may take a while to run (even without REGUL, which I believe should always be turned off in practice). So, at least to develop you code, you could simply use half the metallicities and ages (in a way that you keep the largest metallicity and age), which won't make your performance with pPXF much worse than your current code.

In [12]:
n_models_dev = 6 * 27 * 2
n_models_dev

324

To obtain estimates of ages, metallicities and alpha/Fe, you will need to adapt Cappellari's code to handle the names of the models correctly, get the weights of the fit and obtain averages. This is done in the pPXF tutorial, you only need to add another dimension to the estimates. 